In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from matplotlib import pyplot as plt
from matplotlib import cm

from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, LogisticRegression, SGDClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, PowerTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn import metrics
from sklearn.base import BaseEstimator
from sklearn.svm import SVC, SVR 
from sklearn.metrics import mean_squared_error, f1_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [3]:
cd /Users/pwalesdi/Desktop/GA/NBA_Player_Prediction/

/Users/pwalesdi/Desktop/GA/NBA_Player_Prediction


In [4]:
%store -r nba
%store -r improvement
%store -r per_improvement
%store -r second_year_prime

In [5]:
# Creating a data set to do future predictions that can't be completely tested
test_master = second_year_prime.loc[(second_year_prime['DRAFT_YEAR+1'] == 2017) | (second_year_prime['DRAFT_YEAR+1'] == 2018) | (second_year_prime['DRAFT_YEAR+1'] == 2016)]
test_master

,Player_name,player_id,SEASON,Tm_x,DRAFT_YEAR+1,Draft_team,Pk,Pos,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,College,Yrs,PTS,TRB,AST,FG%,3P%,FT%,PPG,RPG,APG,draft_round,target
4,Bam Adebayo,adebaba01,2019,MIA,2018.0,MIA,14.0,C,21,82,1913,17.9,0.623,0.031,0.465,9.2,24.0,16.6,14.2,1.8,3.0,17.1,15.8,3.4,3.4,6.8,0.171,-0.6,3.6,3.0,2.4,University of Kentucky,2.0,1206.0,978.0,285.0,0.550,0.136,0.728,8.0,6.5,1.9,1.0,0
10,Jarrett Allen,allenja01,2019,BRK,2018.0,BRK,22.0,C,20,80,2096,18.5,0.632,0.079,0.489,9.6,24.0,16.8,7.9,1.0,4.5,13.0,15.9,4.4,3.3,7.6,0.175,-0.7,3.2,2.5,2.4,University of Texas at Austin,2.0,1460.0,1060.0,159.0,0.590,0.183,0.732,9.6,7.0,1.0,1.0,0
11,Kadeem Allen,allenka01,2019,NYK,2018.0,BOS,53.0,SG,26,19,416,16.2,0.574,0.255,0.383,2.0,11.1,6.4,28.7,1.9,0.8,13.6,19.6,0.7,0.2,1.0,0.111,0.5,-1.6,-1.1,0.1,University of Arizona,2.0,208.0,62.0,88.0,0.436,0.362,0.778,5.6,1.7,2.4,2.0,0
18,Ike Anigbogu,anigbik01,2019,IND,2018.0,IND,47.0,C,20,3,6,-6.0,0.000,0.000,0.000,18.9,36.2,27.7,19.4,0.0,14.8,25.0,29.1,-0.1,0.0,-0.1,-0.480,-22.5,8.5,-14.0,0.0,University of California Los Angeles,2.0,13.0,12.0,1.0,0.333,NaN,0.833,0.9,0.9,0.1,2.0,0
22,OG Anunoby,anunoog01,2019,TOR,2018.0,TOR,23.0,SF,21,67,1352,9.8,0.544,0.500,0.153,4.8,10.8,7.9,4.7,1.6,1.3,11.3,15.5,0.3,1.7,2.0,0.070,-1.3,0.1,-1.2,0.3,Indiana University,2.0,907.0,381.0,102.0,0.461,0.351,0.605,6.4,2.7,0.7,1.0,0
29,Dwayne Bacon,bacondw01,2019,CHO,2018.0,NOP,40.0,SG,23,43,759,11.7,0.570,0.339,0.179,1.1,11.7,6.3,9.2,0.8,0.6,6.1,16.8,0.9,0.3,1.2,0.073,-1.2,-2.4,-3.5,-0.3,Florida State University,2.0,491.0,213.0,85.0,0.432,0.377,0.761,5.1,2.2,0.9,2.0,0
35,Lonzo Ball,balllo01,2019,LAL,2018.0,LAL,2.0,PG,21,47,1423,11.7,0.487,0.500,0.105,4.0,14.0,9.1,23.8,2.3,1.1,17.8,16.9,-0.2,1.7,1.6,0.052,-0.9,1.5,0.5,0.9,University of California Los Angeles,2.0,993.0,611.0,631.0,0.380,0.315,0.437,10.0,6.2,6.4,1.0,0
51,Jordan Bell,belljo01,2019,GSW,2018.0,CHI,38.0,C,24,68,788,12.9,0.531,0.010,0.214,7.8,16.7,12.5,12.3,1.2,5.4,16.7,13.7,0.5,1.1,1.6,0.096,-3.1,2.8,-0.4,0.3,University of Oregon,2.0,485.0,391.0,178.0,0.570,0.000,0.647,3.9,3.1,1.4,2.0,0
62,Jaron Blossomgame,blossja01,2019,CLE,2018.0,SAS,59.0,SF,25,27,439,10.2,0.510,0.368,0.123,6.3,19.1,12.4,4.2,0.8,1.6,9.0,12.2,0.2,0.1,0.3,0.030,-3.1,-1.5,-4.5,-0.3,Clemson University,1.0,114.0,98.0,13.0,0.443,0.256,0.769,4.2,3.6,0.5,2.0,0
66,Jonah Bolden,boldejo01,2019,PHI,2018.0,PHI,36.0,PF,23,44,639,12.7,0.595,0.593,0.167,8.0,19.0,13.7,8.5,1.3,4.8,17.2,13.8,0.5,0.9,1.4,0.106,-0.8,2.0,1.2,0.5,NaN,1.0,207.0,165.0,40.0,0.494,0.354,0.481,4.7,3.8,0.9,2.0,0


In [6]:
features = [
            'Pk', 
            'Age', 
            'G', 
            'MP', 
            'TS%', 
            '3PAr', 
            'FTr', 
            'ORB%', 
            'DRB%', 
            'TRB%', 
            'AST%', 
            'BLK%', 
            'TOV%', 
            'USG%', 
#             'OWS', 
#             'DWS', 
            'WS', 
            'WS/48', 
            'OBPM', 
            'DBPM', 
            'BPM', 
            'VORP']

In [9]:
y = second_year_prime['target'] # Setting y --> our target variable
X = second_year_prime[features] # Creating our X variables based on the features we want to use to predict target
ss = StandardScaler() # Instantiate a standard scalar
X_sc = ss.fit_transform(X) # Fit and transform our X variable
ros = RandomOverSampler(random_state=0) # Instantiate a random oversampler in order to oversample our training set
X_resampled, y_resampled = ros.fit_resample(X_sc, y) # Fit that oversampler to our X_sc (scaled) and y data
print(sorted(Counter(y_resampled).items())) # Show the balance btw classes

[(0, 448), (1, 448)]


In [10]:
# Train, test, split based on our oversampled data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=100, test_size=0.25)

In [ ]:
results = pd.DataFrame(lr.predict(X_test), columns=['predicted'])

# Create column for observed values.
y_test = y_test.reset_index()
results['actual'] = y_test['target']
results['index1'] = y_test['index']
results.set_index('index1', inplace=True)
row_ids = results[results['predicted'] != results['actual']].index
results["Player_name"] = ''
for i in row_ids:
    results["Player_name"][i] = (second_year_prime["Player_name"][i])
results.sort_values(by=["actual"], ascending=False)

In [11]:
lr_class = LogisticRegression(penalty='l1', C=40, solver='liblinear')
knn_class = KNeighborsClassifier(n_neighbors=3, p=4, leaf_size=10)
tree_class = DecisionTreeClassifier(max_features='auto', min_samples_leaf=3, min_samples_split=4, random_state=100)
bag_class = BaggingClassifier(bootstrap=False, max_features=8, max_samples=100, n_estimators=100, random_state=100)
forest_class = RandomForestClassifier(bootstrap=True, max_leaf_nodes=None, min_samples_leaf=3, min_samples_split=8, n_estimators=9, random_state=100)
ada_class = AdaBoostClassifier(learning_rate=0.78, n_estimators=100, random_state=100)
svc = SVC(degree=8, C=2.5, gamma=0.1, kernel='poly', random_state=100)
grad_class = GradientBoostingClassifier(n_estimators=100, min_samples_leaf=3, min_samples_split=8, max_depth=3)

In [12]:
class_models = {
            'lr_class': lr_class, 
            'forest_class': forest_class, 
            'tree_class': tree_class,
            'ada_class': ada_class,            
            'knn_class': knn_class, 
            'bag_class': bag_class, 
            'svc': svc,
            "grad": grad_class
                } 
# models_c = [lr_c, knn_c, tree_c, bag_c, forest_c, ada_c, svc]
# model_c_names = ['lr_c', 'knn_c', 'tree_c', 'bag_c', 'forest_c', 'ada_c', 'svc']

In [13]:
y_pred_testc = []
y_pred_trainc = []

for model in class_models.values():
    model.fit(X_train, y_train)
    y_pred_testc.append(model.predict(X_test))
    y_pred_trainc.append(model.predict(X_train))

y_pred_testc_df = pd.DataFrame(y_pred_testc, index=class_models.keys()).T
y_pred_trainc_df = pd.DataFrame(y_pred_trainc, index=class_models.keys()).T
print(y_pred_testc_df.shape)
print(y_pred_trainc_df.shape)

(224, 8)
(672, 8)


In [14]:
accuracy = {'train': [], 'test': [], 'F1-train': [], 'F1-test': [], 'true_neg': [], 'fal_pos': [], 'fal_neg': [], 'true_po': []}
for model in class_models.values():
    accuracy['train'].append(model.score(X_train, y_train))
    accuracy['test'].append(model.score(X_test, y_test))
for col in y_pred_testc_df:
    accuracy['F1-train'].append(f1_score(y_train, y_pred_trainc_df[col]))
    accuracy['F1-test'].append(f1_score(y_test, y_pred_testc_df[col]))
for col in y_pred_testc_df:
    accuracy['true_neg'].append(confusion_matrix(y_test, y_pred_testc_df[col])[0][0])
    accuracy['fal_pos'].append(confusion_matrix(y_test, y_pred_testc_df[col])[0][1])
    accuracy['fal_neg'].append(confusion_matrix(y_test, y_pred_testc_df[col])[1][0])
    accuracy['true_po'].append(confusion_matrix(y_test, y_pred_testc_df[col])[1][1])
    
accuracy_df = pd.DataFrame(accuracy, index=class_models.keys())
accuracy_df

,train,test,F1-train,F1-test,true_neg,fal_pos,fal_neg,true_po
lr_class,0.882440,0.843750,0.889819,0.848485,91,25,10,98
forest_class,0.982143,0.959821,0.982659,0.960000,107,9,0,108
tree_class,0.980655,0.933036,0.981241,0.935065,101,15,0,108
ada_class,0.998512,0.964286,0.998532,0.964286,108,8,0,108
knn_class,0.965774,0.919643,0.967283,0.923077,98,18,0,108
bag_class,0.940476,0.933036,0.944444,0.935065,101,15,0,108
svc,0.992560,0.928571,0.992593,0.928571,104,12,4,104
grad,1.000000,0.964286,1.000000,0.964286,108,8,0,108


In [15]:
master = test_master[features]
master_sc = ss.fit_transform(master)
predicted_player = pd.DataFrame(index=master.index)
predicted_player[['Player_name', 'DRAFT_YEAR+1']] = test_master[['Player_name', 'DRAFT_YEAR+1']]
for (model_name, model) in class_models.items():
    predicted_player[model_name] = model.predict(master_sc)
predicted_player['tot'] = predicted_player['lr_class'] + predicted_player['forest_class'] + predicted_player['tree_class'] + predicted_player['ada_class'] + predicted_player['knn_class'] + predicted_player['bag_class'] + predicted_player['svc'] + predicted_player['grad'] 
predicted_player.sort_values(by='tot', ascending=False)

,Player_name,DRAFT_YEAR+1,lr_class,forest_class,tree_class,ada_class,knn_class,bag_class,svc,grad,tot
1610,Kristaps Porzingis,2016.0,1,1,1,1,1,1,1,1,8
1145,Domantas Sabonis,2017.0,1,1,1,1,1,1,1,1,8
1436,Willy Hernangomez,2016.0,1,1,1,1,1,1,1,1,8
469,Donovan Mitchell,2018.0,1,1,1,1,1,1,1,1,8
1422,Montrezl Harrell,2016.0,1,1,1,1,1,1,1,1,8
222,De'Aaron Fox,2018.0,1,1,1,1,1,1,1,1,8
1300,Devin Booker,2016.0,1,1,1,1,1,1,1,1,8
1448,Richaun Holmes,2016.0,1,1,1,1,1,1,1,1,8
1068,Jamal Murray,2017.0,1,1,1,1,0,1,0,1,6
140,Zach Collins,2018.0,1,1,1,0,0,1,0,1,5


In [ ]:
params = {
    'max_depth' : [3, 4],
    'min_samples_leaf' : [2, 3],
    'min_samples_split' : [6, 7, 8],
    'n_estimators' : [30, 35, 37],
    "learning_rate" : (np.logspace(-1.6, -1, 20))
}
gs = GridSearchCV(
    GradientBoostingClassifier(),
    params,
    cv=3,
    verbose=1,
    return_train_score=False,
    n_jobs=2)
gs.fit(X_train, y_train)
print(gs.best_score_)
print()
print(gs.best_params_)
print()
print(gs.score(X_test, y_test))
pred = model.predict(X_test)
f1_score(y_test, pred)